In [5]:
import os
import torch
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
from glob import glob
import tifffile

# 设置路径
hsv_base_path = "/home/sirui/INNOMAUS/2025_new/gt_b/b/"
dem_path = "/home/sirui/INNOMAUS/2025_new/dem/DEM.tif"
runoff_base_path = "/home/sirui/INNOMAUS/2025_new/berlin_runoff"
output_path = "/home/sirui/INNOMAUS/2025_new/pt"
os.makedirs(output_path, exist_ok=True)

def read_raster(file_path, target_meta=None):
    data=  tifffile.imread(file_path)
    """ 读取栅格数据并根据目标元数据进行重投影 """
    with rasterio.open(file_path) as src:
        crs = src.crs
        meta = src.meta.copy()

        # 处理缺失 CRS 的情况
        if crs is None:
            print(f"Warning: {file_path} has no CRS. Assigning default EPSG:25833.")
            meta['crs'] = "EPSG:25833"  # 你可以根据需要调整 EPSG 代码

        if target_meta:
            target_transform = target_meta['transform']
            target_shape = (target_meta['height'], target_meta['width'])

            destination = np.zeros(target_shape, dtype=np.float32)
            reproject(
                source=data,
                destination=destination,
                src_transform=meta['transform'],
                src_crs=meta['crs'],
                dst_transform=target_transform,
                dst_crs=target_meta['crs'],
                resampling=Resampling.bilinear
            )
            return destination

        return data


# 读取DEM，作为对齐基准
dem_data = read_raster(dem_path)
dem_data = np.nan_to_num(dem_data, nan=75.4477)
with rasterio.open(dem_path) as dem_src:
    dem_meta = dem_src.meta.copy()
    dem_meta.update({"width": 3212, "height": 2727})

# 读取事件数据
event_folders = sorted(glob(os.path.join(hsv_base_path, "*/")))
for event_folder in event_folders:
    event_name = os.path.basename(os.path.dirname(event_folder))
    print(event_name)
    # 获取 H, S, V 文件
    path_h = event_folder+ "*H.tif"
    path_u = event_folder+ "*U.tif"
    path_v = event_folder+ "*V.tif"
    h_files = sorted(glob(path_h))
    s_files = sorted(glob(path_u))
    v_files = sorted(glob(path_v))

    data_tensor = np.zeros((2727, 3212, 25, 5), dtype=np.float32)

    for h_file, s_file, v_file in zip(h_files, s_files, v_files):
        # 解析时间信息
        time_index = int(os.path.basename(h_file).split("_")[-1].replace("H.tif", ""))
        time_index = min(time_index // 300, 24)  # 限制最大索引
        
        # 读取 H, S, V 数据
        h_data = read_raster(h_file, target_meta=dem_meta)
        s_data = read_raster(s_file, target_meta=dem_meta)
        v_data = read_raster(v_file, target_meta=dem_meta)

        # 存入相应位置
        data_tensor[:, :, time_index, 0] = h_data
        data_tensor[:, :, time_index, 1] = s_data
        data_tensor[:, :, time_index, 2] = v_data

    # 读取 runoff 数据
    runoff_folder = os.path.join(runoff_base_path, "b", f"b_{event_name}")
    runoff_files = sorted(glob(os.path.join(runoff_folder, "*.tif")))
    for file in runoff_files:
        time_index = int(os.path.basename(file).split("_")[-1].replace(".tif", ""))
        time_index = min(time_index // 300, 24)
        runoff_data = read_raster(file, target_meta=dem_meta)
        print(runoff_data.max())
        data_tensor[:, :, time_index, 3] = runoff_data[:,:3212]

    # DEM 数据
    data_tensor[:, :, :, 4] = np.repeat(dem_data[:, :, np.newaxis], 25, axis=2)
    print(dem_data[10,10],"dem")
    # 转换为 torch 格式
    torch_tensor = torch.tensor(data_tensor)
    torch.save(torch_tensor, os.path.join(output_path, f"b_{event_name}.pt"))

print("数据处理完成！")

100mm
7.8903174
13.553332
14.13009
16.82366
18.858234
20.339167
21.394156
12.096661
22.137583
22.659803
19.49212
14.824902
11.058647
8.3435335
6.991723
6.0047293
5.187728
4.5070353
13.288333
3.9358923
3.4532766
3.0426865
2.6911175
2.388262
13.513145
75.4477 dem
100y
3.2471077
6.6004553
6.6233797
6.6295657
6.6312323
7.469786
8.300561
5.212707
8.976067
9.516671
8.619169
7.0841084
5.663487
4.5116444
3.6065965
2.898262
2.3408623
2.0545542
6.2127953
1.8236837
1.6204839
1.4411566
1.2837964
1.147988
6.5160675
75.4477 dem
105mm
8.359084
14.232288
15.264409
18.055967
20.136341
21.632092
22.685783
12.781279
23.420826
23.93251
20.516947
15.514035
11.513381
8.652306
7.345034
6.2934484
5.4264092
4.706513
13.977247
4.1043115
3.5968292
3.1661313
2.798152
2.4817874
14.194684
75.4477 dem
10y
1.6817434
3.9969428
4.035208
4.048298
4.052757
4.0542736
4.0547895
2.800889
4.3469257
4.7634454
4.4713373
3.847133
3.2051215
2.6392622
2.1639132
1.771013
1.4475366
1.1808771
3.5783324
0.96029603
0.7995909
0.7124309

In [6]:
import os
import torch
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
from glob import glob
import tifffile

# 设置路径
hsv_base_path = "/home/sirui/INNOMAUS/2025_new/gt_b/e1/"
dem_path = "/home/sirui/INNOMAUS/2025_new/dem/DEM.tif"
runoff_base_path = "/home/sirui/INNOMAUS/2025_new/berlin_runoff"
output_path = "/home/sirui/INNOMAUS/2025_new/pt"
os.makedirs(output_path, exist_ok=True)

def read_raster(file_path, target_meta=None):
    data=  tifffile.imread(file_path)
    """ 读取栅格数据并根据目标元数据进行重投影 """
    with rasterio.open(file_path) as src:
        crs = src.crs
        meta = src.meta.copy()

        # 处理缺失 CRS 的情况
        if crs is None:
            print(f"Warning: {file_path} has no CRS. Assigning default EPSG:25833.")
            meta['crs'] = "EPSG:25833"  # 你可以根据需要调整 EPSG 代码

        if target_meta:
            target_transform = target_meta['transform']
            target_shape = (target_meta['height'], target_meta['width'])

            destination = np.zeros(target_shape, dtype=np.float32)
            reproject(
                source=data,
                destination=destination,
                src_transform=meta['transform'],
                src_crs=meta['crs'],
                dst_transform=target_transform,
                dst_crs=target_meta['crs'],
                resampling=Resampling.bilinear
            )
            return destination

        return data


# 读取DEM，作为对齐基准
dem_data = read_raster(dem_path)
dem_data = np.nan_to_num(dem_data, nan=75.4477)
with rasterio.open(dem_path) as dem_src:
    dem_meta = dem_src.meta.copy()
    dem_meta.update({"width": 3212, "height": 2727})

# 读取事件数据
event_folders = sorted(glob(os.path.join(hsv_base_path, "*/")))
for event_folder in event_folders:
    event_name = os.path.basename(os.path.dirname(event_folder))
    print(event_name)
    # 获取 H, S, V 文件
    path_h = event_folder+ "*H.tif"
    path_u = event_folder+ "*U.tif"
    path_v = event_folder+ "*V.tif"
    h_files = sorted(glob(path_h))
    s_files = sorted(glob(path_u))
    v_files = sorted(glob(path_v))

    data_tensor = np.zeros((2727, 3212, 25, 5), dtype=np.float32)

    for h_file, s_file, v_file in zip(h_files, s_files, v_files):
        # 解析时间信息
        time_index = int(os.path.basename(h_file).split("_")[-1].replace("H.tif", ""))
        time_index = min(time_index // 300, 24)  # 限制最大索引
        
        # 读取 H, S, V 数据
        h_data = read_raster(h_file, target_meta=dem_meta)
        s_data = read_raster(s_file, target_meta=dem_meta)
        v_data = read_raster(v_file, target_meta=dem_meta)

        # 存入相应位置
        data_tensor[:, :, time_index, 0] = h_data
        data_tensor[:, :, time_index, 1] = s_data
        data_tensor[:, :, time_index, 2] = v_data

    # 读取 runoff 数据
    runoff_folder = os.path.join(runoff_base_path, "e1", f"e1_{event_name}")
    runoff_files = sorted(glob(os.path.join(runoff_folder, "*.tif")))
    for file in runoff_files:
        time_index = int(os.path.basename(file).split("_")[-1].replace(".tif", ""))
        time_index = min(time_index // 300, 24)
        runoff_data = read_raster(file, target_meta=dem_meta)
        print(runoff_data.max())
        data_tensor[:, :, time_index, 3] = runoff_data[:,:3212]

    # DEM 数据
    data_tensor[:, :, :, 4] = np.repeat(dem_data[:, :, np.newaxis], 25, axis=2)
    print(dem_data[10,10],"dem")
    # 转换为 torch 格式
    torch_tensor = torch.tensor(data_tensor)
    torch.save(torch_tensor, os.path.join(output_path, f"e1_{event_name}.pt"))

print("数据处理完成！")

100mm
56.495106
24.549698
21.243187
18.260763
15.812165
14.006515
12.6490345
30.982092
11.994479
11.463673
9.520453
7.7622695
6.353607
5.57801
4.931852
4.3790674
3.903409
3.4917755
28.453924
3.1336117
2.820379
2.5451314
2.3021822
2.0868478
27.828745
75.4477 dem
100y
24.396196
9.331846
8.858728
8.154159
7.4300923
6.8055887
6.1893864
15.446902
5.6297436
5.158811
4.4819465
3.7983694
3.1990905
2.697172
2.2828991
2.0590615
1.8719765
1.7052538
9.265518
1.5564642
1.4234129
1.3041611
1.197016
1.1005079
9.313958
75.4477 dem
105mm
59.677265
26.020132
22.391594
19.175919
16.56118
14.649429
13.667724
32.469948
12.920784
12.31828
10.202808
8.30023
6.78327
5.8847265
5.193427
4.6038413
4.097928
3.6612244
30.756033
3.282131
2.9513037
2.6611688
2.4055464
2.1793623
29.735987
75.4477 dem
10y
13.172296
4.183674
4.2689967
4.159396
3.9590256
3.7442095
3.5056362
9.386795
3.2723606
3.062468
2.7263505
2.3753138
2.0542183
1.7734475
1.5323794
1.3267922
1.1516769
1.002277
5.9929743
0.87442476
0.80815274
0.7476346

In [1]:
import os
import torch
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
from glob import glob
import tifffile

# 设置路径
hsv_base_path = "/home/sirui/INNOMAUS/2025_new/gt_b/e2/"
dem_path = "/home/sirui/INNOMAUS/2025_new/dem/DEM.tif"
runoff_base_path = "/home/sirui/INNOMAUS/2025_new/berlin_runoff"
output_path = "/home/sirui/INNOMAUS/2025_new/pt"
os.makedirs(output_path, exist_ok=True)

def read_raster(file_path, target_meta=None):
    data=  tifffile.imread(file_path)
    """ 读取栅格数据并根据目标元数据进行重投影 """
    with rasterio.open(file_path) as src:
        crs = src.crs
        meta = src.meta.copy()

        # 处理缺失 CRS 的情况
        if crs is None:
            print(f"Warning: {file_path} has no CRS. Assigning default EPSG:25833.")
            meta['crs'] = "EPSG:25833"  # 你可以根据需要调整 EPSG 代码

        if target_meta:
            target_transform = target_meta['transform']
            target_shape = (target_meta['height'], target_meta['width'])

            destination = np.zeros(target_shape, dtype=np.float32)
            reproject(
                source=data,
                destination=destination,
                src_transform=meta['transform'],
                src_crs=meta['crs'],
                dst_transform=target_transform,
                dst_crs=target_meta['crs'],
                resampling=Resampling.bilinear
            )
            return destination

        return data


# 读取DEM，作为对齐基准
dem_data = read_raster(dem_path)
dem_data = np.nan_to_num(dem_data, nan=75.4477)
with rasterio.open(dem_path) as dem_src:
    dem_meta = dem_src.meta.copy()
    dem_meta.update({"width": 3212, "height": 2727})

# 读取事件数据
event_folders = sorted(glob(os.path.join(hsv_base_path, "*/")))
for event_folder in event_folders:
    event_name = os.path.basename(os.path.dirname(event_folder))
    print(event_name)
    # 获取 H, S, V 文件
    path_h = event_folder+ "*H.tif"
    path_u = event_folder+ "*U.tif"
    path_v = event_folder+ "*V.tif"
    h_files = sorted(glob(path_h))
    s_files = sorted(glob(path_u))
    v_files = sorted(glob(path_v))

    data_tensor = np.zeros((2727, 3212, 25, 5), dtype=np.float32)

    for h_file, s_file, v_file in zip(h_files, s_files, v_files):
        # 解析时间信息
        time_index = int(os.path.basename(h_file).split("_")[-1].replace("H.tif", ""))
        time_index = min(time_index // 300, 24)  # 限制最大索引
        
        # 读取 H, S, V 数据
        h_data = read_raster(h_file, target_meta=dem_meta)
        s_data = read_raster(s_file, target_meta=dem_meta)
        v_data = read_raster(v_file, target_meta=dem_meta)

        # 存入相应位置
        data_tensor[:, :, time_index, 0] = h_data
        data_tensor[:, :, time_index, 1] = s_data
        data_tensor[:, :, time_index, 2] = v_data

    # 读取 runoff 数据
    runoff_folder = os.path.join(runoff_base_path, "e2", f"e2_{event_name}")
    runoff_files = sorted(glob(os.path.join(runoff_folder, "*.tif")))
    for file in runoff_files:
        time_index = int(os.path.basename(file).split("_")[-1].replace(".tif", ""))
        time_index = min(time_index // 300, 24)
        runoff_data = read_raster(file, target_meta=dem_meta)
        print(runoff_data.max())
        data_tensor[:, :, time_index, 3] = runoff_data[:,:3212]

    # DEM 数据
    data_tensor[:, :, :, 4] = np.repeat(dem_data[:, :, np.newaxis], 25, axis=2)
    print(dem_data[10,10],"dem")
    # 转换为 torch 格式
    torch_tensor = torch.tensor(data_tensor)
    torch.save(torch_tensor, os.path.join(output_path, f"e2_{event_name}.pt"))

print("数据处理完成！")

100mm
6.123077
38.635784
33.68026
26.524813
21.144068
17.500013
14.726334
12.4632015
13.088755
12.361797
10.224605
8.317119
6.805046
5.971728
5.265927
4.66459
4.149061
3.7044208
22.108206
3.3187249
2.9823656
2.6875653
2.4279852
2.1984246
62.25376
75.4477 dem
100y
2.4606922
11.018993
11.72216
10.694223
9.424569
8.3230715
7.331204
5.2522955
6.4890575
5.810343
4.9703298
4.1651754
3.4776616
2.9116879
2.450441
2.182097
1.9799111
1.8003398
10.032491
1.6405693
1.4980913
1.3707114
1.256528
1.1539003
28.973557
75.4477 dem
105mm
6.496782
41.75774
35.90053
27.999638
22.184893
18.299477
15.366976
13.187217
14.122852
13.298568
10.966795
8.899058
7.259561
6.3063602
5.5500374
4.907731
4.3586836
3.8863895
23.297707
3.477701
3.1220849
2.8110473
2.537689
2.2963684
65.51671
75.4477 dem
10y
1.1507697
6.9904532
4.958622
5.0058413
4.748977
4.4229116
4.0684304
2.821117
3.7318192
3.4359198
3.0244114
2.6121576
2.243222
1.925427
1.655663
1.4276975
1.2349886
1.0716343
5.622584
0.93261486
0.8445817
0.7807777
0.72

In [8]:
import os
import torch
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
from glob import glob
import tifffile

# 设置路径
hsv_base_path = "/home/sirui/INNOMAUS/2025_new/gt_b/e3/"
dem_path = "/home/sirui/INNOMAUS/2025_new/dem/DEM.tif"
runoff_base_path = "/home/sirui/INNOMAUS/2025_new/berlin_runoff"
output_path = "/home/sirui/INNOMAUS/2025_new/pt"
os.makedirs(output_path, exist_ok=True)

def read_raster(file_path, target_meta=None):
    data=  tifffile.imread(file_path)
    """ 读取栅格数据并根据目标元数据进行重投影 """
    with rasterio.open(file_path) as src:
        crs = src.crs
        meta = src.meta.copy()

        # 处理缺失 CRS 的情况
        if crs is None:
            print(f"Warning: {file_path} has no CRS. Assigning default EPSG:25833.")
            meta['crs'] = "EPSG:25833"  # 你可以根据需要调整 EPSG 代码

        if target_meta:
            target_transform = target_meta['transform']
            target_shape = (target_meta['height'], target_meta['width'])

            destination = np.zeros(target_shape, dtype=np.float32)
            reproject(
                source=data,
                destination=destination,
                src_transform=meta['transform'],
                src_crs=meta['crs'],
                dst_transform=target_transform,
                dst_crs=target_meta['crs'],
                resampling=Resampling.bilinear
            )
            return destination

        return data


# 读取DEM，作为对齐基准
dem_data = read_raster(dem_path)
dem_data = np.nan_to_num(dem_data, nan=75.4477)
with rasterio.open(dem_path) as dem_src:
    dem_meta = dem_src.meta.copy()
    dem_meta.update({"width": 3212, "height": 2727})

# 读取事件数据
event_folders = sorted(glob(os.path.join(hsv_base_path, "*/")))
for event_folder in event_folders:
    event_name = os.path.basename(os.path.dirname(event_folder))
    print(event_name)
    # 获取 H, S, V 文件
    path_h = event_folder+ "*H.tif"
    path_u = event_folder+ "*U.tif"
    path_v = event_folder+ "*V.tif"
    h_files = sorted(glob(path_h))
    s_files = sorted(glob(path_u))
    v_files = sorted(glob(path_v))

    data_tensor = np.zeros((2727, 3212, 25, 5), dtype=np.float32)

    for h_file, s_file, v_file in zip(h_files, s_files, v_files):
        # 解析时间信息
        time_index = int(os.path.basename(h_file).split("_")[-1].replace("H.tif", ""))
        time_index = min(time_index // 300, 24)  # 限制最大索引
        
        # 读取 H, S, V 数据
        h_data = read_raster(h_file, target_meta=dem_meta)
        s_data = read_raster(s_file, target_meta=dem_meta)
        v_data = read_raster(v_file, target_meta=dem_meta)

        # 存入相应位置
        data_tensor[:, :, time_index, 0] = h_data
        data_tensor[:, :, time_index, 1] = s_data
        data_tensor[:, :, time_index, 2] = v_data

    # 读取 runoff 数据
    runoff_folder = os.path.join(runoff_base_path, "e3", f"e3_{event_name}")
    runoff_files = sorted(glob(os.path.join(runoff_folder, "*.tif")))
    for file in runoff_files:
        time_index = int(os.path.basename(file).split("_")[-1].replace(".tif", ""))
        time_index = min(time_index // 300, 24)
        runoff_data = read_raster(file, target_meta=dem_meta)
        print(runoff_data.max())
        data_tensor[:, :, time_index, 3] = runoff_data[:,:3212]

    # DEM 数据
    data_tensor[:, :, :, 4] = np.repeat(dem_data[:, :, np.newaxis], 25, axis=2)
    print(dem_data[10,10],"dem")
    # 转换为 torch 格式
    torch_tensor = torch.tensor(data_tensor)
    torch.save(torch_tensor, os.path.join(output_path, f"e3_{event_name}.pt"))

print("数据处理完成！")

100mm
1.58389
5.1883807
5.36103
6.8626776
7.448009
9.92843
13.712372
2.6467707
22.342104
62.27979
44.36524
31.246489
21.423805
15.185901
11.121041
8.700575
7.4488587
6.429354
3.4033978
5.5915213
4.896822
4.315872
3.8261437
3.4102235
4.6489506
75.4477 dem
100y
0.5641907
2.3980958
2.5506911
3.216155
3.549016
4.654875
6.4102073
1.1099292
10.393123
29.048225
14.238919
11.304655
8.232938
6.586049
5.352841
4.383638
3.6261997
3.0311184
1.4329076
2.5592713
2.1812177
1.9584583
1.7876158
1.643729
2.0533347
75.4477 dem
105mm
1.694253
5.4616933
5.6350727
7.223243
7.8306513
10.450294
14.43405
2.8205485
23.522243
65.540596
47.559826
33.164886
22.53303
15.847079
11.535976
9.188634
7.838493
6.7444186
3.6005723
5.8492594
5.1098905
4.493698
3.9758463
3.5905368
4.906329
75.4477 dem
10y
0.275944
1.343473
1.4595976
1.8993351
2.155855
2.6621807
3.7645872
0.6109055
6.043082
16.928879
5.6374884
4.602817
3.2359185
2.3860726
2.1017292
1.8753729
1.6658003
1.4788473
0.79210234
1.3148545
1.1720508
1.0479981
0.9402

In [9]:
import os
import torch
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds
from glob import glob
import tifffile

# 设置路径
hsv_base_path = "/home/sirui/INNOMAUS/2025_new/gt_b/r/"
dem_path = "/home/sirui/INNOMAUS/2025_new/dem/DEM.tif"
runoff_base_path = "/home/sirui/INNOMAUS/2025_new/berlin_runoff"
output_path = "/home/sirui/INNOMAUS/2025_new/pt"
os.makedirs(output_path, exist_ok=True)

def read_raster(file_path, target_meta=None):
    data=  tifffile.imread(file_path)
    """ 读取栅格数据并根据目标元数据进行重投影 """
    with rasterio.open(file_path) as src:
        crs = src.crs
        meta = src.meta.copy()

        # 处理缺失 CRS 的情况
        if crs is None:
            print(f"Warning: {file_path} has no CRS. Assigning default EPSG:25833.")
            meta['crs'] = "EPSG:25833"  # 你可以根据需要调整 EPSG 代码

        if target_meta:
            target_transform = target_meta['transform']
            target_shape = (target_meta['height'], target_meta['width'])

            destination = np.zeros(target_shape, dtype=np.float32)
            reproject(
                source=data,
                destination=destination,
                src_transform=meta['transform'],
                src_crs=meta['crs'],
                dst_transform=target_transform,
                dst_crs=target_meta['crs'],
                resampling=Resampling.bilinear
            )
            return destination

        return data


# 读取DEM，作为对齐基准
dem_data = read_raster(dem_path)
dem_data = np.nan_to_num(dem_data, nan=75.4477)
with rasterio.open(dem_path) as dem_src:
    dem_meta = dem_src.meta.copy()
    dem_meta.update({"width": 3212, "height": 2727})

# 读取事件数据
event_folders = sorted(glob(os.path.join(hsv_base_path, "*/")))
for event_folder in event_folders:
    event_name = os.path.basename(os.path.dirname(event_folder))
    print(event_name)
    # 获取 H, S, V 文件
    path_h = event_folder+ "*H.tif"
    path_u = event_folder+ "*U.tif"
    path_v = event_folder+ "*V.tif"
    h_files = sorted(glob(path_h))
    s_files = sorted(glob(path_u))
    v_files = sorted(glob(path_v))

    data_tensor = np.zeros((2727, 3212, 25, 5), dtype=np.float32)

    for h_file, s_file, v_file in zip(h_files, s_files, v_files):
        # 解析时间信息
        time_index = int(os.path.basename(h_file).split("_")[-1].replace("H.tif", ""))
        time_index = min(time_index // 300, 24)  # 限制最大索引
        
        # 读取 H, S, V 数据
        h_data = read_raster(h_file, target_meta=dem_meta)
        s_data = read_raster(s_file, target_meta=dem_meta)
        v_data = read_raster(v_file, target_meta=dem_meta)

        # 存入相应位置
        data_tensor[:, :, time_index, 0] = h_data
        data_tensor[:, :, time_index, 1] = s_data
        data_tensor[:, :, time_index, 2] = v_data

    # 读取 runoff 数据
    runoff_folder = os.path.join(runoff_base_path, "r", f"r_{event_name}")
    runoff_files = sorted(glob(os.path.join(runoff_folder, "*.tif")))
    for file in runoff_files:
        time_index = int(os.path.basename(file).split("_")[-1].replace(".tif", ""))
        time_index = min(time_index // 300, 24)
        runoff_data = read_raster(file, target_meta=dem_meta)
        print(runoff_data.max())
        data_tensor[:, :, time_index, 3] = runoff_data[:,:3212]

    # DEM 数据
    data_tensor[:, :, :, 4] = np.repeat(dem_data[:, :, np.newaxis], 25, axis=2)
    print(dem_data[10,10],"dem")
    # 转换为 torch 格式
    torch_tensor = torch.tensor(data_tensor)
    torch.save(torch_tensor, os.path.join(output_path, f"r_{event_name}.pt"))

print("数据处理完成！")

100mm
1.58389
5.3760233
12.14897
9.169297
7.864298
8.583327
9.047357
3.5504603
58.83708
42.83479
38.0376
26.552694
18.188246
12.788036
9.293809
7.79263
6.5814533
5.6919703
8.53933
4.980961
4.3879724
3.888155
3.4637861
3.1010094
5.653273
75.4477 dem
100y
6.8460436
9.384673
9.630456
8.879454
7.84799
7.0200233
6.815146
4.321034
6.4702077
6.241561
5.5245275
4.6527996
3.858497
3.1927602
2.6487355
2.221732
1.9997832
1.802951
26.980469
1.6281903
1.4727244
1.3340985
1.2101742
1.0991035
9.530255
75.4477 dem
105mm
2.6677213
61.687653
27.54213
25.47774
23.253338
22.56572
20.453726
3.3477724
18.290413
21.79952
20.668352
16.030794
11.84528
8.79264
7.273973
6.180309
5.2795515
4.6270075
6.3295965
4.0939174
3.6419494
3.2557962
2.9238956
2.6369946
5.0166225
75.4477 dem
10y
0.77901477
1.3813232
4.7343354
3.0517695
15.542417
6.060321
5.2776613
1.4499608
5.630624
5.5202847
4.9012847
4.1733894
3.5103729
2.9492052
2.4853961
2.104385
1.7909915
1.5320414
1.4987141
1.3167946
1.1367079
0.98504335
0.8789624
0.81